# Gerenciador de finanças

Bibliotecas e opções

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
from plotly import tools
import plotly.graph_objs as go
from lib import lib as l

In [2]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
lcolors = np.array(['#13E881', '#FF5949', '#FFB84C', '#A7BEFA', '#268040', '#800B00', '#A13808', '#464CC2'])
dcolors = np.array(['#268040', '#800B00', '#A13808', '#464CC2'])

## Leitura dos dados

In [3]:
dinv = pd.read_csv('br_invest.csv')
dinv['Data'] = dinv['Data'].apply(lambda x : dt.datetime.strptime(str(x),'%d/%m/%y').date())
dinv.tail()

,ID,Valor,Data,Status,Tipo
25,5.100,1061.910,2019-12-31,Mes,Nu
26,6.000,846.650,2019-12-31,Mes,Nu
27,100.000,1415.500,2019-12-31,Mes,IVVB
28,101.000,957.000,2019-12-31,Mes,BRAX
29,102.000,1415.500,2019-12-31,Mes,IVVB


In [4]:
dflow = l.calc_flow_gains(dinv)
dflow['Conta'] = dflow['Tipo']
dflow['Conta'] = dflow['Conta'].apply(lambda x : x if x == 'Nu' else 'Easy')
dflow.pop('Tipo')
dflow['Area'] = 'Investimento'

In [5]:
dg = pd.read_csv('br_ganhos.csv')
dg['Data'] = dg['Data'].apply(lambda x : dt.datetime.strptime(str(x),'%d/%m/%y').date())
dg.tail()

,Nome,Valor,Data,Conta,Area
11,Rappi,880.000,2019-11-14,Nu,Salario
12,Rappi,561.600,2019-11-29,Alelo,Salario
13,Rappi,1686.760,2019-11-29,Nu,Salario
14,Rappi,880.000,2019-12-14,Nu,Salario
15,Cash,256.000,2019-12-01,Cash,Pais


In [6]:
dg = pd.concat([dg, dflow], sort=False).reset_index(drop=True)

In [7]:
df = pd.read_csv('br_gastos.csv')
df['Data'] = df['Data'].apply(lambda x : dt.datetime.strptime(str(x),'%d/%m/%y').date())
df.tail(10)

,Nome,Valor,Data,Pagamento,Area
69,Airbnb Foz,527.740,2019-12-23,Nu,Viagem
70,Cafe,75.000,2019-12-31,Pais,Alimentacao
71,Refeicoes em casa,450.000,2019-12-31,Pais,Alimentacao
72,Carro,1000.000,2019-12-31,Pais,Transporte
73,Refeicoes fora,297.000,2019-12-31,Pais,Alimentacao
74,Headset,88.980,2019-12-31,Nu,Eletronicos
75,Agua,72.333,2019-12-31,Pais,Moradia
76,Luz,138.667,2019-12-31,Pais,Moradia
77,Internet,191.333,2019-12-31,Pais,Moradia
78,Fatura,-177.960,2019-12-31,Nu,Credito


In [8]:
dtr = pd.read_csv('br_transf.csv')
dtr.tail()

,De,Para,Valor
0,BB,Nu,12491.070
1,Nu,Easy,1455.000
2,Nu,Easy,731.400
3,Nu,Easy,870.270
4,Nu,Easy,537.000


## Saldo atual

In [9]:
l.calc_funds(dg, df, dtr, [714.01, 0, 219.5, 3741.37, 14894.27])

,Saldo,Lido,Erro
Alelo,619.830,714.010,-94.180
BB,0.000,0.000,0.000
Cash,219.500,219.500,0.000
Easy,3807.070,3741.370,65.700
Nu,14357.620,14894.270,-536.650


## Análise mensal

In [10]:
monthly_earnings = l.window_mean(l.group_sum(dg, group=[]))
monthly_spendings = l.window_mean(l.group_sum(df, group=[]))
monthly_real_spendings = l.window_mean(l.group_sum(df[df['Pagamento'] != 'Pais'], group=[]))

### Balanço

In [11]:
fig = l.plot_bar([monthly_earnings, monthly_spendings, monthly_real_spendings], y='Valor', 
                 color=lcolors[0:3], 
                 name=['Ganho', 'Gasto', 'Gasto Real'])
l.plot_line([monthly_earnings, monthly_spendings, monthly_real_spendings], fig=fig, y='ValorAvg', 
            color=dcolors[0:3], 
            name=['Ganho médio', 'Gasto médio', 'Gasto Real médio'])

FigureWidget({
    'data': [{'hoverinfo': 'y',
              'marker': {'color': '#13E881'},
              'na…

### Saldo

In [12]:
fund = l.calc_balance(monthly_earnings, monthly_real_spendings)
fund = l.window_mean(fund, select='Var')

In [13]:
l.plot_stacked_area(fund, y='Saldo', color=lcolors[3])

FigureWidget({
    'data': [{'marker': {'color': '#A7BEFA'},
              'stackgroup': 'one',
              …

In [14]:
fund['Pct'] = fund['Saldo'].apply(lambda x : 100*(x - fund.head(1)['Saldo']) / fund.head(1)['Saldo'])

In [15]:
l.plot_line(fund, y='Pct', color=dcolors[3])

FigureWidget({
    'data': [{'marker': {'color': '#464CC2'},
              'type': 'scatter',
              'u…

In [16]:
fig = l.plot_bar(fund, y='Var', color=lcolors[3], name='Variacao')
l.plot_line(fund, fig=fig, y='VarAvg', color=dcolors[3], name='Variacao média')

FigureWidget({
    'data': [{'marker': {'color': '#A7BEFA'},
              'name': 'Variacao',
              '…

## Matriz de finanças

In [17]:
df = df[df['Area'] != 'Credito']
area_spendings = l.group_sum(df[df['Pagamento'] != 'Pais'], group=['Area'])
area_spendings_p = l.group_sum(df, group=['Area'])
area_earnings = l.group_sum(dg, group=['Area'])

### Gastos

In [18]:
l.plot_stacked_area(area_spendings, color=lcolors, name=area_spendings.columns)

FigureWidget({
    'data': [{'hoverinfo': 'y',
              'marker': {'color': '#13E881'},
              'na…

In [19]:
l.plot_pie(area_spendings.sum(), colors=lcolors)

FigureWidget({
    'data': [{'hole': 0.4,
              'labels': array(['Alimentacao', 'Banco', 'Eletronicos'…

In [20]:
l.plot_pie(area_spendings_p.sum(), colors=lcolors)

FigureWidget({
    'data': [{'hole': 0.4,
              'labels': array(['Alimentacao', 'Banco', 'Eletronicos'…

### Ganhos

In [21]:
l.plot_stacked_area(area_earnings, color=lcolors, name=area_earnings.columns)

FigureWidget({
    'data': [{'hoverinfo': 'y',
              'marker': {'color': '#13E881'},
              'na…

In [22]:
l.plot_pie(area_earnings.sum(), colors=lcolors)

FigureWidget({
    'data': [{'hole': 0.4,
              'labels': array(['Investimento', 'Pais', 'Salario'], d…